# Gradient-Effected Object Recognition Gauge for hive Entrances (GEORGE)
Neural-net-powered honeybee hive-mounted pollen, varroa, and wasp counter

## Imports

In [1]:
import os
import pathlib
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import host_subplot 
import mpl_toolkits.axisartist as AA
import random
import glob
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
import pandas as pd
import json
from GEORGE_Library import *
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

print("Done")

Using TensorFlow backend
Done


## Define global variables

In [2]:
category_index = {1: {'id': 1, 'name': 'regular'}, 2: {'id': 2, 'name': 'pollen'}, 3: {'id': 3, 'name': 'varroa'}, 4: {'id': 4, 'name': 'wasps'}}
train_augmented_fp = "Dataset/Custom_Dataset/Train"
val_augmented_fp = "Dataset/Custom_Dataset/Validation"
model_directory = 'C:/Users/khard/OneDrive/Documents/GitHub/GEORGE/'
model_name = 'extract_superimp_model'
model_dest = os.path.join(os.sep, model_directory, model_name)
tflite_save = True

# The `label_id_offset` shifts all classes so the model receives one-hot labels where non-background classes start counting at the zeroth index.
label_id_offset = 1
score_threshold = 0.3
batch_size = 64
learning_rate = 0.01
epochs = 5
num_batches = round(len(glob.glob(train_augmented_fp + '/*')) / batch_size) # AKA "Iterations"
num_classes = 4

print("Done")

Done


## Create model and restore weights for all but last layer

This cell builds a single stage detection architecture (RetinaNet) and restores all but the classification layer at the top (which will be automatically randomly initialized).  For simplicity, the image size of 640x640 is hardcoded.

In [3]:
retinapath = pathlib.Path('.').absolute() / "models" / "research" / "object_detection" / "test_data" / "checkpoint"

# Clone the tensorflow models repository if it doesn't already exist
if os.path.exists(retinapath):
    pass
elif not os.path.exists(retinapath):
    print("Doesn't exist")
    # !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
    # !tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
    # !mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

print("Done")

Done


In [4]:
# credit: deeplearning.ai (https://github.com/https-deeplearning-ai)

tf.keras.backend.clear_session()
print('Building model and restoring weights for fine-tuning...', flush=True)
pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
checkpoint_path = 'models/research/object_detection/test_data/checkpoint/ckpt-0'
new_checkpoint_path = './tf_ckpts'

# Load pipeline config and build a detection model.
#
# Since we are working off of a COCO architecture which predicts 90
# class slots by default, we override the `num_classes` field
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(
      model_config=model_config, is_training=True)

# Set up object-based checkpoint restore --- RetinaNet has two prediction
# `heads` --- one for classification, the other for box regression.  We will
# restore the box regression head but initialize the classification head
# from scratch (we show the omission below by commenting out the line that
# we would add if we wanted to restore both heads)
fake_box_predictor = tf.compat.v2.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    # _prediction_heads=detection_model._box_predictor._prediction_heads,
    #    (i.e., the classification head that we *will not* restore)
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
    )
fake_model = tf.compat.v2.train.Checkpoint(
          _feature_extractor=detection_model._feature_extractor,
          _box_predictor=fake_box_predictor)

# Create initial checkpoint with fake model
ckpt = tf.compat.v2.train.Checkpoint(model = fake_model, step = tf.Variable(1))
manager = tf.compat.v2.train.CheckpointManager(ckpt, new_checkpoint_path, max_to_keep = 1)

# Try to restore checkpoint from tf_ckpts folder.
# If none exist, restore from tf models folder
batch_num = 0
epoch_num = 0
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restoring from {}".format(manager.latest_checkpoint))
    trainParameters = json.load(open(new_checkpoint_path + "/trainParameters.txt"))
    batch_num = trainParameters["batch"]
    epoch_num = trainParameters["epoch"]
else:
    ckpt.restore(checkpoint_path).expect_partial()
    print("Initializing from scratch.")

# Run model through a dummy image so that variables are created
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)
print('Weights restored!')

Building model and restoring weights for fine-tuning...
Restoring from ./tf_ckpts\ckpt-37
Weights restored!


## Eager mode custom training loop

In [ ]:
# credit: deeplearning.ai (https://github.com/https-deeplearning-ai)

# Select variables in top layers to fine-tune.
trainable_variables = detection_model.trainable_variables
to_fine_tune = []
prefixes_to_train = [
    'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
    'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']
for var in trainable_variables:
    if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
        to_fine_tune.append(var)

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
train_step_fn = get_model_train_step_function(detection_model, optimizer, to_fine_tune)

loss_array = []
file_list = glob.glob(train_augmented_fp + '/*.png')

coco_annotations_file = train_augmented_fp + "/custom_bee_dataset.json"
coco_images_dir=os.getcwd() + "\\" + train_augmented_fp
coco= COCOParser(coco_annotations_file, coco_images_dir)

start_flag = 1
print('Start fine-tuning!', flush=True)
for epoch in range(epochs - epoch_num):
    p = np.random.permutation(len(file_list))
    train_random_file_list = [file_list[i] for i in p]
    with tqdm(total = num_batches, unit="batch") as pbar:
        pbar.set_description(desc=("Epoch %i" % (epoch_num + 1) + "/%i" % epochs))
        if start_flag:
            pbar.update(batch_num)
            start_flag = 0
        for idx in range(num_batches - batch_num):
            train_images_np = []
            gt_classes_one_hot_tensors = []
            gt_box_tensors = []
            batch_file_list = train_random_file_list[(idx * batch_size):((idx + 1) * batch_size)]
            for filename in batch_file_list:
                temp_train_labels = []
                temp_gt_boxes = []
                
                img_id = int(Path(filename).stem)
                train_images_np.append(np.array(Image.open(filename).convert('RGB')).astype('uint8'))

                ann_ids = coco.get_annIds(img_id)
                img_w, img_h = coco.get_wh(img_id)
                annotations = coco.load_anns(ann_ids)
                for i, ann in enumerate(annotations):
                    _bbox = ann['bbox']
                    x, y, w, h = [int(b) for b in _bbox]
                    class_id = ann["category_id"]
                    class_name = coco.load_cats(class_id)[0]["name"]
                    trans_label = class_id - 1

                    # Start original script again
                    temp_train_labels.append(np.array(tf.one_hot(trans_label, 4)).astype('float32'))
                    temp_gt_boxes.append(np.array(coco_to_rel_yxyx(x, y, w, h, img_w, img_h)).astype('float32'))

                gt_classes_one_hot_tensors.append(tf.stack(temp_train_labels))
                gt_box_tensors.append(tf.stack(temp_gt_boxes))
            '''
            image_tensors: A list of [1, height, width, 3] Tensor of type tf.float32.
                Note that the height and width can vary across images, as they are
                reshaped within this function to be 640x640.
              groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
                tf.float32 representing groundtruth boxes for each image in the batch.
              groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
                with type tf.float32 representing groundtruth boxes for each image in
                the batch.
            '''
            
            train_image_tensors = prep_train_imgs_only(train_images_np)
            # Grab keys for a random subset of examples
            all_keys = list(range(len(train_image_tensors) - 1))
            random.shuffle(all_keys)
            example_keys = all_keys[:batch_size]

            gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
            gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
            image_tensors = [train_image_tensors[key] for key in example_keys]
            
            # Training step (forward pass + backwards pass)
            total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)
            loss_array.append([total_loss, epoch + 1, idx])
            ckpt.step.assign_add(1)

            save_path = manager.save()
            trainParameters = {"batch": idx + 1, "epoch": epoch}
            json.dump(trainParameters, open(new_checkpoint_path + "/trainParameters.txt", "w"))
            pbar.set_postfix(loss=total_loss.numpy())
            pbar.update(1)
    
    print("Saved checkpoint for epoch {}: {}".format(int(ckpt.step), save_path))
print('Done fine-tuning!')

Start fine-tuning!


Epoch 1/5:  66%|██████████████████████████████▉                | 1027/1563 [19:47:17<9:58:25, 66.99s/batch, loss=0.457]

## Save model

In [ ]:
@tf.function(input_signature=[tf.TensorSpec(shape=[None,640,640,3], dtype=tf.float32)])

def detect(input_tensor): # credit: deeplearning.ai (https://github.com/https-deeplearning-ai)
    """Run detection on an input image.

    Args:
    input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.

    Returns:
    A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
      and `detection_scores`).
    """
    preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    prediction_dict = detection_model.predict(preprocessed_image, shapes)
    return detection_model.postprocess(prediction_dict, shapes)

tf.saved_model.save(
    detection_model , model_dest,
    signatures={
      'detect': detect.get_concrete_function()
    })

print("Done")

In [ ]:
if tflite_save:
    # Convert the model
    converter = tf.lite.TFLiteConverter.from_saved_model(model_dest) # path to the SavedModel directory
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    converter._experimental_lower_tensor_list_ops = False
    tflite_model = converter.convert()

    # Save the model.
    with open('model.tflite', 'wb') as f:
        f.write(tflite_model)

print("Done")

In [ ]:
if 'loss_array' in locals() or 'loss_array' in globals():
    y_loss = []
    E_loss = []
    x_loss = []

    for i in loss_array:
        y_loss.append(float(i[0]))
        E_loss.append(i[1])
        x_loss.append(i[2])

    loss_csv = pd.DataFrame()
    loss_csv['y_loss'] = y_loss
    loss_csv['E_loss'] = E_loss
    loss_csv['x_loss'] = x_loss

    print(loss_csv)
    filepath = Path('loss_plot.csv')
    loss_csv.to_csv(filepath)
else:
    loss_array = pd.read_csv('loss_plot.csv')
    y_loss = loss_array["y_loss"]
    E_loss = loss_array["E_loss"]
    x_loss = loss_array["x_loss"]
    
for i in range(len(E_loss)):
    if E_loss[i] == 2:
        x_loss[i] = x_loss[i] + list(E_loss).count(1)
        
print("Done")

In [ ]:
ax1 = host_subplot(111, axes_class=AA.Axes)
plt.title("GEORGE Training Loss vs. Batches")
ax2 = ax1.twiny()
new_fixed_axis = ax2.get_grid_helper().new_fixed_axis

# First X-axis
ax1.plot(x_loss, y_loss)
ax1.set_xticks([int(i) for i in np.linspace(0, len(x_loss), 10)])
ax1.yaxis.set_label_text("Loss")
ax1.xaxis.set_label_text("Batch")

# Second X-axis
offset = 0, -35 # Position of the second axis
ax2.axis["bottom"] = new_fixed_axis(loc="bottom", axes=ax2, offset=offset)
ax2.axis["top"].set_visible(False)

# Need to change this to scale axis divisions dynamically with different number of epochs
ax2.set_xticks([0, list(E_loss).count(1), len(E_loss)])
ax2.xaxis.set_major_formatter(ticker.NullFormatter())
ax2.xaxis.set_minor_locator(ticker.FixedLocator([len(E_loss)/4, (len(E_loss)/4)*3]))
ax2.xaxis.set_minor_formatter(ticker.FixedFormatter(["Epoch 1", "Epoch 2"]))

ax1.grid(1)
plt.subplots_adjust(bottom=0.15)
plt.savefig('GEORGE_Loss_vs_Batch.png')
plt.show()